# Validate Monte Carlo Simulation

This notebook validates the Monte Carlo simulation for scenario generation.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add src directory to path
sys.path.append('../src')

# Set up plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Import Monte Carlo Simulation Module

First, let's import the Monte Carlo simulation module.

In [ ]:
# Import Monte Carlo simulation module
from uncertainty.monte_carlo_sim import (
    run_simulation,
    run_correlated_simulation,
    calculate_scenario_statistics,
    plot_scenarios,
    plot_return_distribution
)

## 2. Single Asset Simulation

Let's run a Monte Carlo simulation for a single asset.

In [ ]:
# Set parameters
S0 = 100.0  # Initial price
mu = 0.0005  # Daily drift (annualized: ~12.6%)
sigma = 0.02  # Daily volatility (annualized: ~31.7%)
days = 30  # Simulation horizon in days
steps_per_day = 10  # Number of steps per day
n_paths = 1000  # Number of paths to simulate

# Run simulation
paths = run_simulation(S0, mu, sigma, days, steps_per_day, n_paths)

# Print simulation shape
print(f"Simulation shape: {paths.shape}")

In [ ]:
# Plot scenarios
fig = plot_scenarios(
    paths,
    n_samples=100,
    quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
    title="Monte Carlo Simulation (30 Days)",
    xlabel="Time Steps",
    ylabel="Price"
)
plt.show()

In [ ]:
# Plot return distribution
fig = plot_return_distribution(
    paths,
    title="Return Distribution (30 Days)",
    xlabel="Return",
    ylabel="Frequency"
)
plt.show()

In [ ]:
# Calculate scenario statistics
stats = calculate_scenario_statistics(paths)

# Print statistics
print(f"Mean return: {stats['mean_return']:.2%}")
print(f"Standard deviation: {stats['std_return']:.2%}")
print(f"95% VaR: {stats['var_95']:.2%}")
print(f"99% VaR: {stats['var_99']:.2%}")
print(f"95% Expected Shortfall: {stats['es_95']:.2%}")
print(f"99% Expected Shortfall: {stats['es_99']:.2%}")

# Print quantiles
print("\nReturn quantiles:")
quantiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]
for i, q in enumerate(quantiles):
    print(f"{q:.0%} quantile: {stats['quantile_returns'][i]:.2%}")

## 3. Multiple Asset Simulation

Now, let's run a Monte Carlo simulation for multiple correlated assets.

In [ ]:
# Set parameters for multiple assets
S0 = [100.0, 150.0, 200.0]  # Initial prices
mu = [0.0005, 0.0007, 0.0003]  # Daily drifts
sigma = [0.02, 0.03, 0.015]  # Daily volatilities

# Correlation matrix
corr_matrix = np.array([
    [1.0, 0.7, 0.3],
    [0.7, 1.0, 0.5],
    [0.3, 0.5, 1.0]
])

# Run correlated simulation
corr_paths = run_correlated_simulation(
    S0, mu, sigma, corr_matrix, days, steps_per_day, n_paths
)

# Print simulation shape
print(f"Correlated simulation shape: {corr_paths.shape}")

In [ ]:
# Plot scenarios for each asset
for i in range(len(S0)):
    fig = plot_scenarios(
        corr_paths[i],
        n_samples=50,
        quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
        title=f"Monte Carlo Simulation - Asset {i+1}",
        xlabel="Time Steps",
        ylabel="Price"
    )
    plt.show()

In [ ]:
# Plot return distributions for each asset
for i in range(len(S0)):
    fig = plot_return_distribution(
        corr_paths[i],
        title=f"Return Distribution - Asset {i+1}",
        xlabel="Return",
        ylabel="Frequency"
    )
    plt.show()

In [ ]:
# Calculate scenario statistics for each asset
for i in range(len(S0)):
    stats = calculate_scenario_statistics(corr_paths[i])
    
    print(f"\nAsset {i+1} Statistics:")
    print(f"Mean return: {stats['mean_return']:.2%}")
    print(f"Standard deviation: {stats['std_return']:.2%}")
    print(f"95% VaR: {stats['var_95']:.2%}")
    print(f"99% VaR: {stats['var_99']:.2%}")

## 4. Analyze Correlations

Let's analyze the correlations between the simulated assets.

In [ ]:
# Calculate returns for each asset
returns = np.zeros((len(S0), n_paths))
for i in range(len(S0)):
    returns[i] = corr_paths[i, :, -1] / corr_paths[i, :, 0] - 1

# Calculate correlation matrix of returns
return_corr = np.corrcoef(returns)

# Print correlation matrices
print("Input correlation matrix:")
print(corr_matrix)
print("\nOutput correlation matrix:")
print(return_corr)

# Plot correlation matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot input correlation matrix
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, ax=axes[0])
axes[0].set_title("Input Correlation Matrix")

# Plot output correlation matrix
sns.heatmap(return_corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1, ax=axes[1])
axes[1].set_title("Output Correlation Matrix")

plt.tight_layout()
plt.show()

## 5. Portfolio Simulation

Let's simulate a portfolio of assets.

In [ ]:
# Define portfolio weights
weights = np.array([0.4, 0.3, 0.3])

# Calculate portfolio value paths
portfolio_paths = np.zeros((n_paths, corr_paths.shape[2]))
for i in range(len(S0)):
    portfolio_paths += weights[i] * corr_paths[i] / S0[i]

# Normalize to start at 100
portfolio_paths *= 100

# Plot portfolio scenarios
fig = plot_scenarios(
    portfolio_paths,
    n_samples=100,
    quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
    title="Portfolio Monte Carlo Simulation",
    xlabel="Time Steps",
    ylabel="Portfolio Value"
)
plt.show()

In [ ]:
# Plot portfolio return distribution
fig = plot_return_distribution(
    portfolio_paths,
    title="Portfolio Return Distribution",
    xlabel="Return",
    ylabel="Frequency"
)
plt.show()

In [ ]:
# Calculate portfolio statistics
portfolio_stats = calculate_scenario_statistics(portfolio_paths)

# Print portfolio statistics
print("Portfolio Statistics:")
print(f"Mean return: {portfolio_stats['mean_return']:.2%}")
print(f"Standard deviation: {portfolio_stats['std_return']:.2%}")
print(f"95% VaR: {portfolio_stats['var_95']:.2%}")
print(f"99% VaR: {portfolio_stats['var_99']:.2%}")
print(f"95% Expected Shortfall: {portfolio_stats['es_95']:.2%}")
print(f"99% Expected Shortfall: {portfolio_stats['es_99']:.2%}")

## 6. Sensitivity Analysis

Let's perform a sensitivity analysis to see how the simulation results change with different parameters.

In [ ]:
# Define parameter ranges
mu_range = np.linspace(0.0, 0.001, 5)  # Daily drift
sigma_range = np.linspace(0.01, 0.03, 5)  # Daily volatility

# Initialize results
results = []

# Run simulations for each parameter combination
for mu in mu_range:
    for sigma in sigma_range:
        # Run simulation
        paths = run_simulation(S0=100.0, mu=mu, sigma=sigma, days=30, steps_per_day=10, n_paths=500)
        
        # Calculate statistics
        stats = calculate_scenario_statistics(paths)
        
        # Store results
        results.append({
            "mu": mu,
            "sigma": sigma,
            "mean_return": stats["mean_return"],
            "std_return": stats["std_return"],
            "var_95": stats["var_95"],
            "var_99": stats["var_99"]
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Display results
results_df

In [ ]:
# Plot sensitivity of mean return to mu and sigma
plt.figure(figsize=(12, 8))

# Reshape data for heatmap
mu_values = results_df["mu"].unique()
sigma_values = results_df["sigma"].unique()
mean_return_matrix = results_df["mean_return"].values.reshape(len(mu_values), len(sigma_values))

# Plot heatmap
sns.heatmap(
    mean_return_matrix,
    annot=True,
    fmt=".2%",
    xticklabels=[f"{sigma:.2%}" for sigma in sigma_values],
    yticklabels=[f"{mu:.2%}" for mu in mu_values],
    cmap="viridis"
)

plt.title("Mean Return Sensitivity")
plt.xlabel("Sigma (Daily Volatility)")
plt.ylabel("Mu (Daily Drift)")
plt.show()

In [ ]:
# Plot sensitivity of 95% VaR to mu and sigma
plt.figure(figsize=(12, 8))

# Reshape data for heatmap
var_95_matrix = results_df["var_95"].values.reshape(len(mu_values), len(sigma_values))

# Plot heatmap
sns.heatmap(
    var_95_matrix,
    annot=True,
    fmt=".2%",
    xticklabels=[f"{sigma:.2%}" for sigma in sigma_values],
    yticklabels=[f"{mu:.2%}" for mu in mu_values],
    cmap="coolwarm_r"
)

plt.title("95% VaR Sensitivity")
plt.xlabel("Sigma (Daily Volatility)")
plt.ylabel("Mu (Daily Drift)")
plt.show()

## 7. Save Simulation Results

Let's save the simulation results for later use.

In [ ]:
# Create results directory if it doesn't exist
os.makedirs("../results", exist_ok=True)

# Save simulation results
np.save("../results/single_asset_paths.npy", paths)
np.save("../results/multi_asset_paths.npy", corr_paths)
np.save("../results/portfolio_paths.npy", portfolio_paths)

# Save sensitivity analysis results
results_df.to_csv("../results/sensitivity_analysis.csv", index=False)

print("Simulation results saved to ../results/ directory")

## 8. Summary

In this notebook, we have validated the Monte Carlo simulation for scenario generation. We have:

1. Run a Monte Carlo simulation for a single asset
2. Run a Monte Carlo simulation for multiple correlated assets
3. Analyzed the correlations between the simulated assets
4. Simulated a portfolio of assets
5. Performed a sensitivity analysis
6. Saved the simulation results for later use

The Monte Carlo simulation provides a powerful tool for scenario generation and risk management in financial applications.